In [4]:
import json

with open('embeddings_output.json') as f:
    embed= json.load(f)

In [8]:
embed[0]

{'text': 'Política de reembolso: Política de reembolso Política de Reembolsos, Cambios y Devoluciones New Line Chile SpA | Kreadores.cl Última actualización: 11-06-2025 En kreadores.cl nos tomamos en serio la protección de tus derechos como consumidor. Por eso, esta política ha sido elaborada conforme a lo establecido en la Ley N° 19.496 sobre Protección de los Derechos de los Consumidores de Chile y en cumplimiento con los requisitos de Google Merchant Center. Esta política aplica a todas las compras realizadas a través de nuestro sitio web: https://www.kreadores.cl 1. Derecho a Retracto Según lo dispuesto en el artículo 3 bis de la Ley del Consumidor chilena, si compraste un producto por internet y fue entregado a domicilio, tienes derecho a retractarte de la compra dentro de un plazo de 30 días corridos desde la compra, siempre que: El producto no haya sido usado, abierto ni dañado. Esté en su empaque original, sellado y en perfectas condiciones. Se acompañe con la boleta o comproba

In [5]:
len(embed[2]['embedding'])

384

# Generación de embeddings para sistema RAG
- **Objetivo**: Entregar embeddings generados a AI Devs para que lo implemente en sistema RAG del agente eCommerce
- **Proceso**: realizar desde el proceso de Chunking, evaluar los documentos generados para elegir el modelo, generar el embedding, evaluar resultados con implementación de Rag con OpenAI.

## Chunking
**Requisitos**
- *Tamaño óptimo:* 200-500 tokens por chunk
- *Overlap:* 50-100 tokens entre chunks consecutivos
- *Estrategias:*
  - Por párrafos (para políticas)
  - Por producto individual (para catálogos) (ya lo maneja equipo de AI devs)
  - Por pregunta-respuesta (para FAQs)

- Metadatos propuestos:
  - politica de reembolso, politica de envio
  -  Secciones: 

In [ ]:
from docx import Document
import json

doc= Document('docs/Recursos.docx')



In [23]:
# Cada párrafo
doc.paragraphs[3].text

'En kreadores.cl nos tomamos en serio la protección de tus derechos como consumidor. Por eso, esta política ha sido elaborada conforme a lo establecido en la Ley N° 19.496 sobre Protección de los Derechos de los Consumidores de Chile y en cumplimiento con los requisitos de Google Merchant Center.'

In [ ]:
# Extracción del texto
parrafos= []
for texto in doc.paragraphs:
    if texto.text: # si hay texto en la línea
        parrafos.append(texto.text.strip())

# Lo unimos en un solo string
texto_completo = "\n".join(parrafos)


Chunking - División en fragmentos

In [65]:
import re

# division por titulos numerados y separadores
chunks= re.split(r"(?:\n_{5,}\n|\n{2,}|(?=\n\d\.))",texto_completo)
chunks = [chunk.strip() for chunk in chunks if chunk.strip()]



In [ ]:
len(chunks)

15

In [101]:
tamanio= []
for i in chunks:
    tamanio.append(len(i))
print(f'Máxima longitud de chunk generado: {max(tamanio)}')

Máxima longitud de chunk generado: 697


In [66]:
chunks

['Política de reembolso:\nPolítica de reembolso\nPolítica de Reembolsos, Cambios y Devoluciones\nNew Line Chile SpA | Kreadores.cl\nÚltima actualización: 11-06-2025\nEn kreadores.cl nos tomamos en serio la protección de tus derechos como consumidor. Por eso, esta política ha sido elaborada conforme a lo establecido en la Ley N° 19.496 sobre Protección de los Derechos de los Consumidores de Chile y en cumplimiento con los requisitos de Google Merchant Center.\nEsta política aplica a todas las compras realizadas a través de nuestro sitio web: https://www.kreadores.cl',
 '1. Derecho a Retracto\nSegún lo dispuesto en el artículo 3 bis de la Ley del Consumidor chilena, si compraste un producto por internet y fue entregado a domicilio, tienes derecho a retractarte de la compra dentro de un plazo de 30 días corridos desde la compra, siempre que:\nEl producto no haya sido usado, abierto ni dañado.\nEsté en su empaque original, sellado y en perfectas condiciones.\nSe acompañe con la boleta o co

Se añade la metadata

In [ ]:
import re


politicas= { 'Politica de reembolso':['1. Derecho a Retracto',
 '2. Garantía Legal por Producto Defectuoso',
 '3. Cambios Voluntarios',
 '4. Productos Excluidos de Reembolso o Cambio',
 '5. Procedimiento de Devolución',
 '6. Reembolsos','7. Costos de Envío en Devoluciones',
 '8. Dudas y Contacto'], 
 'Politica de envio': [ '1. Despachos a Nivel Nacional',
 '2. Plazos de despacho y entrega',
 '3. Horario de corte para procesamiento de pedidos',
 '4. Seguimiento de envío',
 '5. Responsabilidad del cliente',
 '6. Problemas con la entrega']}

chunks_metadata= []
for chunk in chunks:
    match_seccion= re.search(r"^\d+\.\s+.+", chunk)
    if match_seccion:
        seccion= match_seccion.group(0)
    else:
        seccion='General'

    for politica, secciones in politicas.items():
        if seccion in secciones:
            politica_metadata= politica
            break
        else:
            politica_metadata='Otros'

    # agrega la metadata
    chunks_metadata.append( {
        'text': chunk,
        'metadata': {
            'politica':politica_metadata,
            'seccion': seccion
        }
    })


In [99]:
chunks_metadata

[{'text': 'Política de reembolso:\nPolítica de reembolso\nPolítica de Reembolsos, Cambios y Devoluciones\nNew Line Chile SpA | Kreadores.cl\nÚltima actualización: 11-06-2025\nEn kreadores.cl nos tomamos en serio la protección de tus derechos como consumidor. Por eso, esta política ha sido elaborada conforme a lo establecido en la Ley N° 19.496 sobre Protección de los Derechos de los Consumidores de Chile y en cumplimiento con los requisitos de Google Merchant Center.\nEsta política aplica a todas las compras realizadas a través de nuestro sitio web: https://www.kreadores.cl',
  'metadata': {'politica': 'Otros', 'seccion': 'General'}},
 {'text': '1. Derecho a Retracto\nSegún lo dispuesto en el artículo 3 bis de la Ley del Consumidor chilena, si compraste un producto por internet y fue entregado a domicilio, tienes derecho a retractarte de la compra dentro de un plazo de 30 días corridos desde la compra, siempre que:\nEl producto no haya sido usado, abierto ni dañado.\nEsté en su empaque

## Implementación del modelo

Características principales del documento:
- Texto
- Español

In [ ]:
from fastembed import TextEmbedding

# modelos disponibles
modelos= TextEmbedding.list_supported_models()
modelos

In [113]:
for modelo in modelos:
    if 'English'not in modelo['description']:
        print(modelo['model'],modelo['description'], '\n',modelo['dim'], '\n')

BAAI/bge-small-zh-v1.5 Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year. 
 512 

sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 Text embeddings, Unimodal (text), Multilingual (~50 languages), 512 input tokens truncation, Prefixes for queries/documents: not necessary, 2019 year. 
 384 

sentence-transformers/paraphrase-multilingual-mpnet-base-v2 Text embeddings, Unimodal (text), Multilingual (~50 languages), 384 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year. 
 768 

intfloat/multilingual-e5-large Text embeddings, Unimodal (text), Multilingual (~100 languages), 512 input tokens truncation, Prefixes for queries/documents: necessary, 2024 year. 
 1024 

jinaai/jina-embeddings-v3 Multi-task unimodal (text) embedding model, multi-lingual (~100), 1024 tokens truncation, and 8192 sequence length. Prefixes for queries/documents: not necessary, 2024 year. 
 1024 



Como solicitan una dimensionalidad de 1536, y como los vectores y la query deben tener la mismas dimensiones. Entonces se descarta uso de FastEmbed. 
Se procede a evaluar OpenAi

OPENAI

In [116]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
api_key=os.getenv('OPENAI_API_KEY')
cliente= OpenAI()
cliente

In [ ]:
response = cliente.embeddings.create(
    input= chunks_metadata
)

In [ ]:
# from openai import OpenAI
# client = OpenAI()

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-small"
# )

# print(response.data[0].embedding)